In [119]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

# Load the dataset
data = pd.read_csv('Churn_Modelling.csv')
# preprocessing data
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

data.head()



,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [120]:
lable_encoder_gender = LabelEncoder()
data["Gender"]
print(data["Gender"].unique())
data['Gender']= lable_encoder_gender.fit_transform(data['Gender'])
print("Encoder classes:", lable_encoder_gender.classes_)
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(lable_encoder_gender, file)


['Female' 'Male']
Encoder classes: ['Female' 'Male']


In [121]:
# One hot ecoding categorical variables
from sklearn.preprocessing import OneHotEncoder

print(data.columns)

oneHotEncoder_geography= OneHotEncoder()
print(type(data['Geography']))
print(type(data[['Geography']]))
geo_encoder = oneHotEncoder_geography.fit_transform(data[['Geography']]).toarray()

geo_df = pd.DataFrame(geo_encoder, columns=oneHotEncoder_geography.get_feature_names_out(['Geography']))
geo_df.head()

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [122]:
data= pd.concat([data.drop('Geography',axis=1), geo_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [123]:

# save files

with open('one_hot_encoder_geography.pkl', 'wb') as file:
    pickle.dump(oneHotEncoder_geography, file)

In [124]:
X= data.drop('Exited', axis=1)
Y= data['Exited']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [125]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [126]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense  
import tensorflow

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

model.summary()
# Compile the model

adam = tensorflow.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])




Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                832       
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [127]:
from tensorflow.keras.callbacks import EarlyStopping
import datetime
#log directory = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [128]:
#Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [129]:
# train the model
history = model.fit(X_train, Y_train, validation_data=(X_test,Y_test) , epochs=100, 
                    callbacks=[tensorboard_callback, early_stopping])

model.save('model.h5')

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.4497 - accuracy: 0.8100 - val_loss: 0.3961 - val_accuracy: 0.8260
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3847 - accuracy: 0.8425 - val_loss: 0.3610 - val_accuracy: 0.8520
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3554 - accuracy: 0.8568 - val_loss: 0.3478 - val_accuracy: 0.8575
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3444 - accuracy: 0.8587 - val_loss: 0.3433 - val_accuracy: 0.8635
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3391 - accuracy: 0.8630 - val_loss: 0.3435 - val_accuracy: 0.8585
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3360 - accuracy: 0.8618 - val_loss: 0.3429 - val_accuracy: 0.8620
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3334 - accuracy: 0.8629 - val_loss: 0.3463 - val_accuracy: 0.8600

/Users/hemantapaudel/python/ML/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [130]:
# load tensorboard
%load_ext tensorboard
%tensorboard --logdir logs/fit



The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 44243), started 1 day, 0:31:50 ago. (Use '!kill 44243' to kill it.)